In [1]:
import smtplib
import numpy as np
import pandas as pd

In [12]:
def send_email(user, pwd, recipient, subject, body):
    import smtplib

    gmail_user = user
    gmail_pwd = pwd
    FROM = user
    TO = recipient if type(recipient) is list else [recipient]
    SUBJECT = subject
    TEXT = body

    # Prepare actual message
    message = """\From: %s\nTo: %s\nSubject: %s\n\n%s
    """ % (FROM, ", ".join(TO), SUBJECT, TEXT)
    try:
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.ehlo()
        server.starttls()
        server.login(gmail_user, gmail_pwd)
        server.sendmail(FROM, TO, message)
        server.close()
        print 'successfully sent the mail'
    except:
        print "failed to send mail"
        
def gradeStatus(grade):
    grade = float(grade)*100.0
    if grade >= 90.0:
        return ', which means you are doing great! Keep up the good work!'
    elif grade < 90.0 and grade >= 80.0:
        return ', which means your are doing well.'
    elif grade < 80.0 and grade >= 70.0:
        return ', which means you are doing ok, but could improve a bit.'
    elif grade < 70.0:
        return ', which means you are having some trouble, and we should talk if you want.'
    else:
        return '.'

def informalgreet():
    i = np.random.random_integers(0, high=4)
    greet = ['I hope life is treating you well.', 'How are you doing?', 
             'I am glad that you are in my class.', 'How\'s it going?', 
             'I hope things are going well for you.']
    return greet[i]

def signOff():
    i = np.random.random_integers(0, high=5)
    signOffs = ['Best,','Thanks,', 'Take care,', 'Regards,', 'Thank You,', 'Sincerely,']
    return signOffs[i]

def findMissing(x):
    missing = str(x).split(',')
    missing = [i.strip(' ') for i in missing]
    if len(missing) == 0 or missing[0] == '':
        return ' You are not missing any assignments! '
    elif len(missing) == 1:
        return ' You are only missing the {:} assignment.'.format(missing[0])
    elif len(missing) == 2:
        message = ' You are missing your {:} and {:} assignments. You can find links to these on Canvas or talk to me about getting a copy.'.format(missing[0],missing[1])
        return message
    elif len(missing) > 2:
        messageBeg = ' You are missing your '
        messageMid = ', '.join(map(str, missing[0:-1]))
        message = messageBeg + messageMid + ', and ' + missing[-1] + ' assignements.  You can find links to these on Canvas or talk to me about getting a copy.'
        return message
    else:
        return ''

In [13]:
rootsource = r"C:\Users\Brooke\Downloads\20_Nov_14-15_Grades-GEO-1010-009-F16.csv"

ledger = pd.read_csv(rootsource, na_values=['(read only)','NaN'], skiprows=[1])
ledgerlist = list(ledger.iloc[0,:])

participation = [u'Extra Credit: Student Presentation (up to 15 pts) (3128892)',
       u'Participation Points (3128901)', u'Extra Credit: Field Trip (15 pnts) (3128891)',
       u'Extra Credit: Concept Checks (up to 15 pts) (3128890)',
       u'Geology Knowledge Survey (3128882)', u'What do you want to learn? (3128888)',
       u'Roll Call Attendance (3338449)', u'Oil and Coal Extra Credit (3419237)']
eportfolio = [u'Site Selection (3128905)',
       u'List of five sources (3128897)', u'Site Visit Form (3128906)',
       u'Report (3128902)', u'Eportfolio Post (3128889)']
inclasstests = [u'Test 1 (3128907)', u'Test 2 (3128908)', u'Test 3 (3128909)']
canvastests = [u'Test 1 Canvas (3128886)', u'Test 2 Canvas (3128883)', u'Test 3 Canvas (3128875)']
inclassassgn = [u'Ch. 2, 13, & 14: Plate Tectonics (3128873)',
       u'Ch 3: Minerals (3128877)', u'Ch 4 and 5: Igneous Rock and Volcanoes (3128871)',
       u'Ch 6 and 7: Weathering and Sedimentary Rock (3128872)', u'Ch 8: Metamorphic Rock (3128884)',
       u'Ch 9 and 22: Geologic Time (3128878)', u'Ch 10: Deformation (3128879)', u'Ch 11: Earthquakes (3128880)',
       u'Ch 18 and 21: Glaciers and Climate Change (3128887)', u'Ch 16 and 17: Streams and groundwater (3128870)',
       u'Ch 15: Landslides (3128868)']
canvasassgn = [u'Minerals In-class Assignment (3128899)', u'Igneous Rocks In-class Assignment (3128896)',
       u'Sedimentary Rocks In-class Assignment (3128904)', u'Metamorphic Rocks In-class Assignment (3128898)',
       u'Fossils (3128893)', u'Geologic Maps (3128894)', u'Earthquakes (3135773)', u'Water Lab (3128895)',
       u'Stratigraphic Laws (3135624)']

gradedict = {'Participation':[participation,0.1],'Eportfolio':[eportfolio,0.15],'In-class Tests':[inclasstests,0.2],
             'Canvas Tests':[canvastests,0.2],'In-class Assignments':[inclassassgn,0.20],
             'Canvas Assingments':[canvasassgn,0.15]}

def gradeSum(df, name):
    if " Tests" in name and len(gradedict[name][0])>2:
        df[name + ' Points'] = df[gradedict[name][0]].sum(axis=1) - df[gradedict[name][0]].min(axis=1)
    else:
        df[name + ' Points'] = df[gradedict[name][0]].sum(axis=1)
    df[name + ' Percent'] = df[name + ' Points'].apply(lambda x: round(x/df[name + ' Points'][0],2),1)
    df[name + ' Part'] = df[name + ' Percent']*gradedict[name][1] 
    return df

for key in gradedict:
    ledger = gradeSum(ledger, key)
    
import re

ledger['missing'] = ''
for i in range(len(ledger)):
    miss = []
    for col in list(ledger.columns)[6:]:
        
        if ledger.ix[0,col] > 0 and '(' in col:
            if ledger.ix[i,col] > 0:
                pass
            else:
                miss.append(col)
    miss = ','.join(miss)
    miss = re.sub(r'\(.*?\)', '', miss)
    ledger.ix[i,'missing'] = str(miss)

partgrades = []
for col in list(ledger.columns):
    if " Part" in col:
        partgrades.append(col)    
ledger['final'] = ledger[partgrades].sum(axis=1)

ledger['lastName'] = ledger['Student'].apply(lambda x: x.split(' ')[0][:-1],1)
ledger['firstName'] = ledger['Student'].apply(lambda x: x.split(' ')[1],1)
ledger['missing'] = ledger['missing'].apply(lambda x: findMissing(x),1)

In [14]:
def boost(x,name):
    if x < 0.7:
        return 'The {:} portion of your grade is lower than average, and we should work on it to boost your grade.'.format(name)
    else:
        return ''
ledger['boost'] = ledger['Participation Percent'].apply(lambda x: boost(x,'Participation Points'))

In [15]:
for i in range(len(ledger)):
    name = ledger['firstName'][i]
    studPoints = ledger['final'][i]*100
    missing = ledger['missing'][i]
    gradestatus = gradeStatus(ledger['final'][i])
    email = str(ledger['SIS Login ID'][i])+'@slcc.edu'
    #missing = findMissing(x)
    print email
    boost = ledger['boost'][i]
    personalMess = ''
    body = ('''{:},\n     {:} This is just a routine report informing you of your grade to help you do well in my class. You have a {:}% in my class{:} {:}{:} {:}\n\n{:}\nPaul'''.format(name, informalgreet(), studPoints, gradestatus, missing, boost, personalMess, signOff()))
    print(body)

nan@slcc.edu
,
     I hope life is treating you well. This is just a routine report informing you of your grade to help you do well in my class. You have a 100.0% in my class, which means you are doing great! Keep up the good work!  You are not missing any assignments! 

Sincerely,
Paul
badams44@slcc.edu
Bentley,
     How are you doing? This is just a routine report informing you of your grade to help you do well in my class. You have a 0.0% in my class, which means you are having some trouble, and we should talk if you want.  You are missing your Participation Points, Geology Knowledge Survey, What do you want to learn?, Roll Call Attendance, Site Selection, List of five sources, Test 1, Test 2, Test 1 Canvas, Test 2 Canvas, Ch. 2, 13, & 14: Plate Tectonics, Ch 3: Minerals, Ch 4 and 5: Igneous Rock and Volcanoes, Ch 6 and 7: Weathering and Sedimentary Rock, Ch 8: Metamorphic Rock, Ch 9 and 22: Geologic Time, Ch 10: Deformation, Ch 11: Earthquakes, Minerals In-class Assignment, Igneous